<a href="https://colab.research.google.com/github/phon1e/political_bias_on_fake_news/blob/main/project_source_code_24_CID276227.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"Impact of Political Bias on Fake News
Detection and Bias Mitigation" project source code




### DATA COLLECTION

In [ ]:
import pandas as pd
from datasets import load_dataset
import seaborn as sns
import matplotlib.pyplot as plt

 ## DATASET (LIAR + Buzzfeed)

In [ ]:
#LIAR dataset
def read_dataframe(tsv_file:str)->pd.DataFrame:
    """read lair dataset and convert to dataframe

    Args:
        tsv_file (string): tsv file path

    Returns:
        dataframe: dataframe of the lair dataset
    """
    df = pd.read_csv(tsv_file, delimiter='\t', dtype=object)
    df.fillna("", inplace=True)
    df.columns = [
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.

        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.

        'context' # Column 14: the context (venue / location of the speech or statement).
    ]
    return df


#Buzzfeed dataset
def xml_to_df(f_path:str)->pd.DataFrame:
    """read all xml files in the folder and convert to dataframe by tag
    <mainText>, <orientation>, <title>, <veracity>

    Args:
        f_path (string): folder path

    Returns:
        pd.DataFrame: dataframe of the xml files
    """
    all_files = os.listdir(f_path)
    data = []
    for file in all_files:
        tree = ET.parse(f_path + '\\' + file)
        root = tree.getroot()
        main_text = root.find('mainText').text
        orientation = root.find('orientation').text
        title = root.find('title').text
        veracity = root.find('veracity').text
        data.append([title, main_text, orientation, veracity])
    df = pd.DataFrame(data, columns=['title', 'content', 'bias', 'label'])
    return df

In [ ]:
fp = 'liar_ds/'
liar_df = read_dataframe(f'{fp}/train.tsv')

In [ ]:
f_path = "\articles"
buzzfeed_bias_df = xml_df = xml_to_df(f_path)

## DATA PREPROCESSING

In [ ]:
def label_bar_chart(input_df: pd.DataFrame, title: str = "LIAR Dataset") -> None:
    """plot lair dataset as bar chart

    Args:
        input_df (pd.DataFrame): _description_
        title (str, optional): _description_. Defaults to "LIAR Dataset".

    Returns:
        _type_: _description_
    """
    # computes frequencies of labels and converts to percentages
    label_frequencies = input_df['label'].value_counts(normalize=True)

    def multiply_100(x):
        return x * 100

    # "apply" is a handy way to call a function on every row of data.
    label_frequencies = label_frequencies.apply(multiply_100)

    # bar chart ordering and  colors for readability.
    labels = ['pants-fire', 'false', 'barely-true', 'half-true', 'mostly-true', 'true']
    colors = [
        'orangered', # pants-fire
        'coral', # false
        'salmon', # barely-true
        'peachpuff', # half-true
        'skyblue', # mostly-true
        'deepskyblue' # true
    ]

    label_frequencies = label_frequencies.reindex(index = labels)


    # creates a horizontal bar chart with a descriptive title
    axis = label_frequencies.plot(kind='barh', figsize=(4, 2), color=colors)
    axis.set_title(f"distribution of label values ({title}, sample_size={len(input_df)})", size=20);

In [ ]:
#mapping label between two dataset

label_mapping = {
    "pants-fire": "false",
    "false": "false",
    "barely true": "mixed",
    "half-true": "mixed",
    "mostly-true": "true",
    "true": "true"
}

news_liar_bias_df = liar_df.copy()
news_liar_bias_df['bias'] = "unk"
news_liar_bias_df['harmonized_label'] = news_liar_bias_df['label'].map(label_mapping)

label_mapping_bf = {
    "mostly true": "true",
    "mixture of true and false": "mixed",
    "mostly false": "false",
    "no factual content": "false"
}

buzzfeed_bias_df['harmonized_label'] = buzzfeed_bias_df['label'].map(label_mapping_bf)

#rename buzzfeed_bias_df columns content to statement
buzzfeed_bias_df.rename(columns={'content': 'statement'}, inplace=True)

buzzfeed_bias_df.harmonized_label.value_counts()

In [ ]:
#handle missing value
buzzfeed_bias_df = buzzfeed_bias_df.dropna()

#combine liar and buzzfeed dataset statement, harmonized_label and bias
news_liar_bias_df = news_liar_bias_df[['statement', 'harmonized_label', 'bias']]
buzzfeed_bias_df = buzzfeed_bias_df[['statement', 'harmonized_label', 'bias']]

#combine liar and buzzfeed dataset
combined_df = pd.concat([news_liar_bias_df, buzzfeed_bias_df])
#drop na
combine_df = combined_df.dropna()

## TRAINING AND CLASSIFIER

### Bag of words classifier

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
#bias clf 0,1,2 mainstream, left, right

#bias clf
def bow_clf(df, col1, col2):
   """classifier using bag of words

    Args:
        df (pd.DataFrame): _description_
        col1 (str): column target1
        col2 (str): column target2

    Returns:
        classifer and count vectorizer
    """
    #split data
    X_train, X_test, y_train, y_test = train_test_split( df[col1], df[col2], test_size=0.2, random_state=42)

    #count vectorizer
    count_vectorizer = CountVectorizer()
    X_train_counts = count_vectorizer.fit_transform(X_train)
    X_test_counts = count_vectorizer.transform(X_test)

    #naive bayes
    clf = MultinomialNB()
    clf.fit(X_train_counts, y_train)

    #predict
    pred = clf.predict(X_test_counts)

    #classification report
    print(classification_report(y_test, pred))
    return clf, count_vectorizer

### Train and evaluate function

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def train_and_evaluate(model,tokenizer, df, col1,col2, savename):
   """train and evaluate model

    Args:
        model : transformer-based model
        tokenizer : transformer-based tokenizer
        df: dataframe
        col1: column target1
        col2: column target2
        savename: name of the saved model
    Returns:
        model and tokenizer

    """
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(device)

    # Ensure the correct columns are used
    df = df[[col1, col2]]

    # Split data into training and validation sets
    train_texts, val_texts, train_labels, val_labels = train_test_split(df[col1].tolist(), df[col2].tolist(), test_size=0.2, random_state=42)

    # Tokenize the data
    train_encodings = tokenizer(train_texts, truncation=True, padding=True)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True)

    # Create a PyTorch Dataset
    class BiasDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

    train_dataset = BiasDataset(train_encodings, train_labels)
    val_dataset = BiasDataset(val_encodings, val_labels)

    # Load the model and set up training
    model = model
    model.to(device)
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=4,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        gradient_accumulation_steps=2,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        fp16=True
    )

    def compute_metrics(p):
        preds = p.predictions.argmax(-1)
        labels = p.label_ids
        acc = accuracy_score(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
        return {
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    #save the model
    model.save_pretrained(f'saved_model\\{savename}')
    tokenizer.save_pretrained(f'saved_model\\{savename}')

    # Evaluate the model
    results = trainer.evaluate()

    # Function to make predictions
    def predict(texts):
        model.eval()  # Set the model to evaluation mode
        encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model(**encodings)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        return predictions.tolist()

    return results, predict

## MODEL

### Roberta model

In [ ]:
def get_ids_mask(sentences):
     """
    Get input ids and attention masks for a list of sentences.
    Args:
        sentences (list): List of sentences.
    Returns:
        roberta_input_ids (torch.Tensor): Tensor of input ids.
        roberta_attention_masks (torch.Tensor): Tensor of attention masks.
        label (torch.Tensor): Tensor of labels.
        sent_ids (torch.Tensor): Tensor of sentence ids.

    """

    max_len = 0

    for sent in sentences:
        input_ids = roberta_tokenizer.encode(sent, add_special_tokens=True)
        max_len = max(max_len, len(input_ids))

    roberta_input_ids = []
    roberta_attention_masks = []
    sents_ids = []
    counter = 0

    for sent in sentences:
        roberta_encoded_dict = roberta_tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            truncation=True,
            max_length=64,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        roberta_input_ids.append(roberta_encoded_dict['input_ids'])
        roberta_attention_masks.append(roberta_encoded_dict['attention_mask'])

        #collect ids
        sents_ids.append(counter)
        counter += 1

    #cvt to tensor
    roberta_input_ids = torch.cat(roberta_input_ids, dim=0)
    roberta_attention_masks = torch.cat(roberta_attention_masks, dim=0)

    label=torch.tensor(labels)
    sent_ids = torch.tensor(sents_ids)

    return roberta_input_ids, roberta_attention_masks, label, sent_ids

roberta_input_ids, roberta_attention_masks, label, sent_ids = get_ids_mask(sentences)

### DistilBERT model and Bag of Words model (political bias)

In [ ]:

results_bias_clf_distilbert, predict_distilbert = train_and_evaluate(
    AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3),
    AutoTokenizer.from_pretrained('distilbert-base-uncased'),
    bias_clf_df,
    'statement',
    'bias',
    savename='bias_distil_model_2'
)

clf_fake_news_df, vectorizer_bow = bow_clf(combined_bias_fact_df, 'statement', 'bias')

### DistilBERT model and Bag of Words model (fake news)

In [ ]:
model_fake_news = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels= num_labels)
tkn = AutoTokenizer.from_pretrained('distilbert-base-uncased')
results_fakeN_clf_distilbert, predict_distilbert_FN = train_and_evaluate(
    AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2),
    AutoTokenizer.from_pretrained('distilbert-base-uncased'),
    true_n_false_df,
    'statement',
    'fake_news',
    savename='bias_distil_model_2'
)

clf_fakeN_df, vectorizer_FN = bow_clf(combined_bias_fact_df, 'statement', 'fake_news')

### MASKING MODEL (bias classifier with attention score)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import nltk
from nltk.corpus import stopwords
import spacy

nltk.download('stopwords')

def detect_and_mask(sentence, model, tokenizer, nlp, top_k=0.2):
    """
    Detect and mask named entities in a sentence based on attention scores.

    Args:
        sentence (str): Input sentence.
        model (torch.nn.Module): Language model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer.
        nlp (spacy.lang.en.English): spaCy English language model.
        top_k (float): Proportion of tokens to mask.

    Returns:
        str: Masked sentence.


    """

    stop_words = set(stopwords.words('english'))
    additional_avoid_words = set(['.', ',', '!', '?', '(', ')', "'", '"', 'is', 'am', 'are', 'was', 'were',
                                  'be', 'being', 'been', 'am', 'do', 'does', 'did',
                                  'have', 'has', 'had', '', ' '])
    avoid_words = stop_words.union(additional_avoid_words)
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt',
                       truncation=True, padding=True,
                        max_length=512)

    # Ensure the model outputs attentions
    outputs = model(**inputs, output_attentions=True)

    # Extract attention scores
    attentions = outputs.attentions[-1]  # Get attention from the last layer

    # Average attention scores across heads and tokens
    attention_scores = attentions.mean(dim=1).squeeze(0).mean(dim=0)  # Average over heads and batch dimension, keep token dimension

    # Get the tokenized input tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze())

    # Use spaCy NER to identify named entities
    doc = nlp(sentence)
    ner_indices = set()

    # Manually map character-level positions to token-level
    token_offsets = []
    current_char_index = 0
    for token in tokens:
        if token in ['[CLS]', '[SEP]', '[PAD]', '<mask>']:
            token_offsets.append((current_char_index, current_char_index))
            continue
        current_char_index = sentence.find(token.replace('##', ''), current_char_index)
        token_offsets.append((current_char_index, current_char_index + len(token.replace('##', ''))))
        current_char_index += len(token.replace('##', ''))

    for ent in doc.ents:
        for idx, (start, end) in enumerate(token_offsets):
            if start >= ent.start_char and end <= ent.end_char:
                ner_indices.add(idx)

    # Ignore special tokens, stopwords, and additional avoid words
    valid_indices = [i for i, token in enumerate(tokens) if token.lower() not in avoid_words and token not in ['[CLS]', '[SEP]', '[PAD]', '<mask>']]

    # Ensure NER indices are considered in valid indices
    valid_indices = list(set(valid_indices + list(ner_indices)))

    # Filter attention scores for valid tokens
    filtered_attention_scores = attention_scores[valid_indices]
    filtered_tokens = [tokens[i] for i in valid_indices]

    # Identify top-k% tokens based on filtered attention scores
    num_tokens_to_mask = int(len(filtered_tokens) * top_k)
    top_k_indices = torch.topk(filtered_attention_scores, num_tokens_to_mask, largest=True).indices.tolist()

    # Create a masked sentence
    masked_tokens = tokens.copy()
    for idx in top_k_indices:
        masked_tokens[valid_indices[idx]] = '<mask>'

    masked_sentence = tokenizer.convert_tokens_to_string(masked_tokens)

    return masked_sentence

### FILL TEXT MODEL (Trained BART)

In [ ]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments, GenerationConfig

def tokenize_function(examples):
    """
    Tokenize function for the BART model.

    Args:
        examples (dict): Input examples.

    Returns:
        dict: Tokenized examples.
    """
    inputs = tokenizer(examples["masked_statement"], max_length=1024, truncation=True, padding="max_length")
    targets = tokenizer(examples["statement"], max_length=1024, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

dataset = Dataset.from_pandas(df_mainstream)
dataset = Dataset.from_pandas(df_mainstream2)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

trainer.train()

# Define generation configuration
generation_config = GenerationConfig(
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)

# Save generation configuration
generation_config.save_pretrained("/bart_model_mainstream")

# Export model and tokenizer
model.save_pretrained("/bart_model_mainstream")
tokenizer.save_pretrained("/bart_tkn_mainstream")


### PARAPHRASE MODEL

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)

tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

def get_response(input_text,num_return_sequences,num_beams):
    """
    Get a response from model (PEGASUS).

    Args:
        input_text (str): input text
        num_return_sequences (int): number of return sequences
        num_beams (int): number of beams

    Returns:
        str: response from model
    """
  batch = tokenizer([input_text],truncation=True,padding=True,max_length=60, return_tensors="pt")
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

def paraphrase_txtv2(statement):
    """
    Paraphrase a statement using the Pegasus model.

    Args:
        statement (str): Input statement.

    Returns:
        str: Paraphrased statement.
    """
    paraphrased = get_response(input_text=statement, num_return_sequences=1, num_beams=1)[0]
    sentences = paraphrased.split('. ')
    return sentences[0].strip() + '.'

## REGENERATE FUNCTION (BART)

In [ ]:
def generate_text_batch(masked_sents, model, tokenizer, grad_=False, num_iterations=3, learning_rate=1e-5, alpha= 0.1):
     """
    Generate text using the BART model.
    Args:
        masked_sents (list): List of masked sentences.
        model (torch.nn.Module): BART model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer.

    Returns:
        list: List of generated sentences.


    """


    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    inputs = tokenizer(masked_sents, return_tensors='pt', max_length=100, truncation=True, padding=True).to(device)

    if grad_:
        model.train()  # Enable training mode to allow gradient updates
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        for _ in range(num_iterations):
            optimizer.zero_grad()

            # Forward pass
            outputs = model(**inputs, labels=inputs['input_ids'])
            loss = outputs.loss

            # Compute neutralization loss
            logits = outputs.logits
            softmax_logits = F.softmax(logits, dim=-1)
            uniform_dist = torch.full_like(softmax_logits, 1.0 / softmax_logits.size(-1))
            neutralization_loss = F.kl_div(softmax_logits.log(), uniform_dist, reduction='batchmean')

            # Total loss with balancing
            total_loss = loss + alpha * neutralization_loss
            total_loss.backward()

            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update model parameters
            optimizer.step()

    model.eval()  # Disable training mode for generation
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100, num_beams=4, no_repeat_ngram_size=2, early_stopping=True)

    neutral_sents = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return neutral_sents



## EVALUATION

In [ ]:
def predict_text(input_text, model, tokenizer, classes_label, max_length=512):
    """
    Predict the class of the input text using the provided model and tokenizer.

    Args:
        input_text (str): Input text for prediction.
        model (torch.nn.Module): Language model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer.
        classes_label (list): List of class labels.
        max_length (int): Maximum sequence length.

    Returns:
        str: Predicted class label.


    """
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    pred_class = torch.argmax(logits, dim=-1).item()

    return classes_label[pred_class]

In [ ]:
bleu_metric = load_metric("sacrebleu")

def calculate_bleu(statement, generated_statement):
    """
    Calculate the BLEU score between the original statement and the generated statement.

    Args:
        statement (str): Original statement.
        generated_statement (str): Generated statement.

    Returns:
        float: BLEU score.

    """
    result = bleu_metric.compute(predictions=[generated_statement], references=[[statement]])
    return result['score']

## EXTRA

In [ ]:
import re
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def find_common_context_words_and_generate_wordclouds(dataframe, text_column, bias_column, bias_types, stop_words, num_common_words=50, max_words_cloud=100):

    """
    Find the most common context-specific words for each bias type and generate word clouds.

    Args:
        dataframe (pandas.DataFrame): Input DataFrame.
        text_column (str): Name of the text column.
        bias_column (str): Name of the bias column.
        bias_types (list): List of bias types.

    Returns:
        dict: Dictionary containing common context-specific words for each bias type.


    """
    def tokenize(text):
        text = text.lower()  # Convert to lowercase
        tokens = re.findall(r'\b\w+\b', text)  # Extract words
        return tokens

    # Dictionary to store common context-specific words for each bias type
    common_words_by_bias = {}

    for bias in bias_types:
        # Tokenize statements by bias type, filtering out possible null values
        bias_tokens = dataframe[dataframe[bias_column] == bias][text_column].dropna().apply(tokenize).sum()

        # Filter out all stop words from tokens
        context_words_filtered = [word for word in bias_tokens if word not in stop_words]

        # Get the most common context-specific words for the current bias type
        common_context_words = Counter(context_words_filtered).most_common(num_common_words)

        # Store the result in the dictionary
        common_words_by_bias[bias] = common_context_words

        # Generate word cloud
        wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=max_words_cloud).generate_from_frequencies(dict(common_context_words))

        # Display the word cloud using matplotlib
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(f'Word Cloud for {bias} Bias')
        plt.axis('off')
        plt.show()

    return common_words_by_bias

# Define stop words manually
manual_stop_words = set([
    'the', 'a', 'to', 'and', 'of', 'in', 'that', 'is', 's','t', 'it', 'for', 'on', 'with', 'as', 'was', 'by', 'he', 'at',
    'from', 'his', 'her', 'an', 'be', 'this', 'which', 'or', 'we', 'you', 'but', 'not', 'are', 'i', 'they', 'have',
    'has', 'had', 'do', 'does', 'did', 'can', 'could', 'will', 'would', 'shall', 'should', 'may', 'might', 'must', 'am',
    'if', 'then', 'them', 'there', 'their', 'were', 'been', 'so', 'what', 'when', 'where', 'who', 'whom', 'why', 'how', 'cls'
])

def print_bias_metrics(y_true, y_pred):
    """
    Print bias metrics including accuracy, precision, recall, and F1 score.

    Args:
        y_true (list): List of true labels.
        y_pred (list): List of predicted labels.

    Returns:
        None


    """


    y_pred_mapped = ['NaN' if pred == 'center' else pred for pred in y_pred]

    # Filter out "center" predictions from both true and predicted labels
    y_true_filtered = [true for true, pred in zip(y_true, y_pred_mapped) if pred != 'NaN']
    y_pred_filtered = [pred for pred in y_pred_mapped if pred != 'NaN']

    # Print metrics
    print('Accuracy:', accuracy_score(y_true_filtered, y_pred_filtered))
    print('Precision:', precision_score(y_true_filtered, y_pred_filtered, average='macro', zero_division=0))
    print('Recall:', recall_score(y_true_filtered, y_pred_filtered, average='macro', zero_division=0))
    print('F1:', f1_score(y_true_filtered, y_pred_filtered, average='macro', zero_division=0))
    print()
    print('Classification Report:')
    print(classification_report(y_true_filtered, y_pred_filtered, target_names=['left', 'right'], zero_division=0))



import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu_nltk(reference, candidate):
    """
    Calculate the BLEU score using NLTK.

    Args:
        reference (str): Reference text.
        candidate (str): Candidate text.

    Returns:
        float: BLEU score.

    """
    reference_tokens = [reference.split()]
    candidate_tokens = candidate.split()


    smoothing_function = SmoothingFunction().method1

    # Calculate BLEU score with smoothing function
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothing_function) * 100
    return bleu_score


def calculate_confidence_scores(model, tokenizer, dataframe, col1, col2):
    """
    Calculate confidence scores for each statement in the DataFrame.

    Args:
        model (torch.nn.Module): Language model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer.
        dataframe (pandas.DataFrame): Input DataFrame.

    Returns:
        pandas.DataFrame: DataFrame with confidence scores.
    """

    # Tokenize the neutral statements
    neutral_statements = dataframe[col1].tolist()
    inputs = tokenizer(neutral_statements, return_tensors="pt", padding=True, truncation=True)

    # Get predictions and raw logits from the model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Convert logits to probabilities using softmax
    probabilities = torch.softmax(logits, dim=-1)

    # Get the predicted class and confidence score for each statement
    predicted_classes = torch.argmax(probabilities, dim=1)
    confidence_scores = probabilities.max(dim=1).values

    # Map predicted classes to their corresponding labels
    class_mapping = {0: 'mainstream', 1: 'left', 2: 'right'}
    dataframe['predicted_class'] = predicted_classes.numpy()
    dataframe['predicted_class'] = dataframe['predicted_class'].map(class_mapping)

    # Add confidence scores to the DataFrame
    dataframe['confidence_score'] = confidence_scores.numpy()

    # Compare the predicted class with the original pred_gen_bias
    dataframe['is_correct'] = dataframe['predicted_class'] == dataframe[col2]

    return dataframe


#XAI (explainable ai)

def predict_proba(texts):
    """
    Predict the probabilities of each class for the given texts (bias).
    Args:
        texts (list): List of input texts.

    Returns:
        numpy.ndarray: output of attention score.

    """
    inputs = ft_fake_news_tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    outputs = ft_fake_news_model(**inputs)

    logits = outputs.logits
    probs = logits.softmax(dim=1).detach().numpy()

    return probs

def predict_proba_bias(texts):
    """
    Predict the probabilities of each class for the given texts(fake news).
    Args:
        texts (list): List of input texts.

    Returns:
        numpy.ndarray: output of attention score.
    """

    inputs = ft_bias_tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    outputs = ft_bias_model(**inputs)

    logits = outputs.logits
    probs = logits.softmax(dim=1).detach().numpy()

    return probs


model and dataset: https://1drv.ms/f/c/00c8e7e1fdd53826/EkeR2y6DghlFuTATk1J289EBpXURIzMah_vyGRGAc6vSfA?e=av7mcV

-buzzfeed dataset: https://zenodo.org/records/1239675

-LIAR dataset : https://paperswithcode.com/dataset/liar